# Inverse location
### PyRugged API - tutorial 3

The aim of this tutorial is to compute the inverse location of a point on Earth in order to give the sensor pixel, with the associated line, seeing this point.

We will also explain how to find the date at which sensor sees a ground point, which is a kind of inverse location only focusing on date.

---

Module imports

In [1]:
import orekit_jcc
import numpy as np 

import os 
import sys


from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.frames import FramesFactory
from org.orekit.time import AbsoluteDate
from org.orekit.time import TimeScalesFactory
from org.orekit.utils import AngularDerivativesFilter
from org.orekit.utils import CartesianDerivativesFilter
from org.orekit.utils import IERSConventions

from pyrugged.configuration.init_orekit import init_orekit
from pyrugged.intersection.algorithm_id import AlgorithmId
from pyrugged.intersection.intersection_algorithm import create_intersection_algorithm
from pyrugged.bodies.body_rotating_frame_id import BodyRotatingFrameId
from pyrugged.bodies.ellipsoid_id import EllipsoidId
from pyrugged.model.inertial_frame_id import InertialFrameId
from pyrugged.model.pyrugged_builder import PyRuggedBuilder
from pyrugged.line_sensor.line_sensor import LineSensor
from pyrugged.line_sensor.linear_line_datation import LinearLineDatation
from pyrugged.los.fixed_rotation import FixedRotation
from pyrugged.los.los_builder import LOSBuilder
from pyrugged.location.optical import OpticalLocation

# add helpers modules from tests
import os
import sys
module_path = os.path.join(os.getcwd(), "./../tests/")
if module_path not in sys.path:
    sys.path.append(module_path)

import helpers

### Initialization 

initialize JVM or GraalVM subtrat VM and initialize Orekit, using internal orekit-data folder

In [2]:
init_orekit()

## 1 - Sensor's definition

### Line of sight
The raw viewing direction of pixel i with respect to the instrument is defined by the vector:

In [3]:
raw_dirs = []
for i in range(2000):
    raw_dirs.append(
        Vector3D(0.0, i * float(np.radians(20)) / 2000.0, 1.0)
    )

The instrument is oriented 10° off nadir around the X-axis. 
We need to rotate the viewing direction to obtain the line of sight in the satellite frame.

In [4]:
los_builder = LOSBuilder(raw_dirs)
los_builder.add_ti_los_transform(
    FixedRotation(
        "10-degrees-rotation",
        Vector3D.PLUS_I,
        float(np.radians(10))
    )
)

line_of_sight = los_builder.build()

### Datation model
We use Orekit for handling time and dates, and PyRugged for defining the datation model:

In [5]:
gps = TimeScalesFactory.getGPS()
abs_date = AbsoluteDate("2009-12-11T16:59:30.0", gps)
line_datation = LinearLineDatation(
    abs_date, 1.0, 20
)

### Line sensor
With the LOS and the datation now defined, we can initialize a line sensor object in PyRugged:

In [6]:
line_sensor = LineSensor(
    "my-sensor",
    line_datation,
    Vector3D.ZERO,
    line_of_sight,
)

## 2 - Satellite position, velocity and attitude

### Reference frames
In our application, we simply need to know the name of the frames we are working with. Positions and
velocities are given in the ITRF terrestrial frame, while the quaternions are given in EME2000
inertial frame.

In [7]:
eme_2000 = FramesFactory.getEME2000()

# We don't want to compute tiny tidal effects at millimeter level
simple_eop = True

itrf = FramesFactory.getITRF(
    IERSConventions.IERS_2010, simple_eop
)

### Satellite attitude

In [8]:
satellite_q_list = []

helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T16:58:42.592937", -0.340236, 0.333952, -0.844012, -0.245684)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T16:59:06.592937", -0.354773, 0.329336, -0.837871, -0.252281)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T16:59:30.592937", -0.369237, 0.324612, -0.831445, -0.258824)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T16:59:54.592937", -0.3836, 0.319792, -0.824743, -0.265299)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:00:18.592937", -0.397834, 0.314883, -0.817777, -0.271695)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:00:42.592937", -0.411912, 0.309895, -0.810561, -0.278001)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:01:06.592937", -0.42581, 0.304838, -0.803111, -0.284206)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:01:30.592937", -0.439505, 0.299722, -0.795442, -0.290301)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:01:54.592937", -0.452976, 0.294556, -0.787571, -0.296279)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:02:18.592937", -0.466207, 0.28935, -0.779516, -0.302131)

### Positions and velocities

In [9]:
satellite_pv_list = []

helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:58:42.592937", -726361.466, -5411878.485, 4637549.599, -2463.635, -4447.634, -5576.736)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:59:04.192937", -779538.267, -5506500.533, 4515934.894, -2459.848, -4312.676, -5683.906)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:59:25.792937", -832615.368, -5598184.195, 4392036.13, -2454.395, -4175.564, -5788.201)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:59:47.392937", -885556.748, -5686883.696, 4265915.971, -2447.273, -4036.368, -5889.568)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:00:08.992937", -938326.32, -5772554.875, 4137638.207, -2438.478, -3895.166, -5987.957)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:00:30.592937", -990887.942, -5855155.21, 4007267.717, -2428.011, -3752.034, -6083.317)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:00:52.192937", -1043205.448, -5934643.836, 3874870.441, -2415.868, -3607.05, -6175.6)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:01:13.792937", -1095242.669, -6010981.571, 3740513.34, -2402.051, -3460.291, -6264.76)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:01:35.392937", -1146963.457, -6084130.93, 3604264.372, -2386.561, -3311.835, -6350.751)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:01:56.992937", -1198331.706, -6154056.146, 3466192.446, -2369.401, -3161.764, -6433.531)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:02:18.592937", -1249311.381, -6220723.191, 3326367.397, -2350.574, -3010.159, -6513.056)

## 3 - PyRugged initialization

In [10]:
builder = PyRuggedBuilder()

builder.set_ellipsoid(
    new_ellipsoid = None,
    ellipsoid_id = EllipsoidId.WGS84,
    body_rotating_frame_id = BodyRotatingFrameId.ITRF,
)

builder.set_time_span(
    abs_date,
    abs_date.shiftedBy(60.0),
    0.01,
    5 / line_sensor.get_rate(0)
)

builder.set_trajectory(
    satellite_pv_list,
    4, CartesianDerivativesFilter.USE_P,
    satellite_q_list, 
    4, AngularDerivativesFilter.USE_R,
    inertial_frame_id=InertialFrameId.EME2000,
)

builder.add_sensor(line_sensor)

rugged = builder.build()

## 4 - Intersection initialization

In [11]:
intersection_algorithm = create_intersection_algorithm(AlgorithmId.IGNORE_DEM_USE_ELLIPSOID)


location = OpticalLocation(rugged, intersection_algorithm)

## 5 - Inverse location of a geodetic point

Point defined by its latitude, longitude and altitude. __Inverse location takes point coordinates as arguments, not the point itself.__

The inverse location will give the sensor pixel number and the associated line number seeing the point on ground. In case the point cannot be seen between the prescribed line numbers, the return result is `None`. No exception will be thrown in this particular case.

For a geodetic point, angles are given in radians and altitude in meters.

In [12]:
latitude = np.array([float(np.radians(37.5849))])
longitude = np.array([float(np.radians(-96.9492))])
altitude = np.array([0.0])

Search the sensor pixel seeing point

In [13]:
#Interval of lines where to search the point
min_line = 0
max_line = 100

line, pixel = location.inverse_location(
    min_line, 
    max_line,
    latitude, 
    longitude, 
    altitude,
)

print(
    f'{"Result : line number "}{line[0]}{" pixel number "}{pixel[0]}'
)

Result : line number 0.0358326202592814 pixel number 0.012435517984594963


## 6 - Date location

Once Rugged initialized, one can compute the date at which sensor sees a point on Earth.

__Date location takes point coordinates as arguments, not the point itself.__

In [14]:
date_line = location.date_location(
    min_line, 
    max_line,
    latitude, 
    longitude, 
    altitude,
)

print(
    f'{"Result : "}{date_line[0]}'
)

Result : 2009-12-11T16:59:14.95179163062503Z


## Inverse location and date location without altitude

__Inverse location__

One can compute the line number and the pixel number of a point defined solely by its latitude en longitude. The altitude will be determined automatically with the DEM : 

In [15]:
line_without_altitude,pixel_without_altitude = location.inverse_location(
    min_line, 
    max_line,
    latitude,
    longitude, 
)

print(
    f'{"Result : line number "}{line_without_altitude}{" pixel number "}{pixel_without_altitude}'
)

Result : line number [np.float64(0.0358326202592814)] pixel number [0.012435517984594963]


__Date location__

Date location for a point defined solely by its latitude en longitude (altitude determined automatically with the DEM):

In [16]:
date_line_without_altitude = location.date_location(
    min_line, 
    max_line, 
    latitude, 
    longitude,
    sensor_name = line_sensor.name
)

print(
    f'{"Result : "}{date_line_without_altitude[0]}'
)

Result : 2009-12-11T16:59:14.95179163062503Z
